# Using Health Inspection Scores to Predict NYC Restaurant Yelp Ratings

## Table of Contents:
Notebook 01_data_processing:
1. [Problem Statement](#Problem-Statement)
2. [Executive Summary](#Executive-Summary)
3. [Data Dictionary](#Data-Dictionary)
4. [Data Import](#Data-Import)
5. [Data Cleaning](#Data-cleaning)

Notebook 02_eda_and_modeling:
1. [Import Libraries](#Import-Libraries)
2. [Load In Dataset](Load-In-Dataset)
3. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
4. [Modeling](#Modeling)
5. [Confusion Matrix](#Confusion-Matrix)
1. [Conclusions and Recommendations](#Conclusions-and-Recommendations)
2. [References](#References)

## Problem Statement
Every restaurant in New York City is scheduled for an unannounced inspection at least once a year. Inspectors from the NYC Department of Health and Mental Hygiene (DOHMH) conducts checks for compliance with city and state food safety regulations and marks points for any condition that violates these rules.

Can the DOHMH inspection results and demographics data be used to predict whether a restaurant has a favorable or unfavorable Yelp review?

## Executive Summary
The Health Department inspects approximately 27,000 restaurants in New York City to monitor their compliance with food safety regulations. Inspectors observe how food is prepared, served and stored and whether restaurant workers are practicing good hygiene. Sanitary violations are issued when the safety of the food being prepared and served is threatened. Sanitary violations displayed in red text are the most critical violations. Examples of sanitary violations include food being held at an unsafe temperature and evidence of mice. Sanitary violations are scored and contribute to a restaurant’s grade. No other violations are scored or contribute to the grade. Examples of non-scored violations include failing to display the Health Department-issued permit and not posting the restaurant's letter grade.

The basic data consists of restaurant inspection results for inspections taking place in the year 2016, including type, action, violation code and descriptions (hereinafter referred to as “NYC inspections”). This data is made available by the NYC’s Department of Health and Mental Hygiene (DOHMH) on NYC OpenData.

The Yelp business information for restaurants represented in the NYC inspections dataset was obtained using Yelp’s Fusion API. Although NYC assigns a unique City-wide Agency Management Information System (CAMIS) number to each restaurant, this number did not provide a link that could identify a corresponding business profile on Yelp. Therefore, we queried the Yelp API for the restaurant's profile by searching the name and address for the restaurant provided in the NYC inspections dataset. Then, we conducted several checks to ensure the returned result was a matching profile, including ensuring the zip codes matched and manually checking restaurants where the Yelp and NYC Inspections restaurant names or addresses had a Levenshtein distance greater than zero.

## Data Dictionary
| Parameter | Description | Data Type | Example |
| ------ | ------ | ------- | ------ |
|camis | DOHMH RecordID number | Integer | 30075445 |
|dba | Doing Buisness As | String | "MORRIS PARK BAKE SHOP" |
|boro | New York City Borough | String | "BRONX" |
|zip_code | zip code | Integer | 10462 |
|cuisine_description| Description of cuisine | String | "Bakery"|
|inspection_date | Date of Inspection | String | "2015-09-04" |
|inspection_year | Year of inspection | String | "2015.0" |
|inspection_month | Month of inspection | String | "9.0" |
|score | Inspection score | String | "6.0" |
|grade | Inspection grade | String | A | 
|name | Restaurant name | String | "Morris Pk Bake Shop" |
|latitude | latitude of the coordinates for the restaurant | String | 40.848446 |
|longitude | longitude of the coordinates for the restaurant | String | -73.856079 |
|review_count | number of reviews for the restaurant | Integer | 31 |
|price | Restaurant price range per person | String | 2 |
|rating | Restaurant Yelp rating | Integer | 4.5 |
|categories | Restaurant categories | String | "['bakeries', 'desserts']" |
|city | City | String | "Bronx" |
|violation_code | DOHMH violation codes issued to restaurant | String | "[06C]" |

## Data Import

### Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import distance
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.api import OLS

pd.options.mode.chained_assignment = None
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

%matplotlib inline

### Load Datasets

In [2]:
# bring in each data set
PATH_YELP_DATA = '../data/yelp_data.csv'
PATH_INSPECTIONS = '../data/nyc_restaurant_inspection_data.csv'
PATH_DEMOGRAPHICS = '../data/demographics_zipcode.csv'

Here the datasets are merged into a single dataset

In [3]:
def load_inspections (url=PATH_INSPECTIONS):
    """Creates DF from NYC inspections data and handles preliminary
    data-cleaning. Sets CAMIS as index.

    Raises:
        RuntimeError if number of rows after cleaning data are fewer than 150K.
    """
    df = pd.read_csv(url)
    # Remove records not containing a zip code and ensure zip code is an int.
    col=np.array(df['ZIPCODE'], np.int64)
    df['ZIPCODE']=col
    #col=np.array(df['column_name'], np.int16)
    if len(df.index) < 150000:
        raise RuntimeError('Inspections DF contains < 150,000 rows.')
    df.index.name = None
    df.rename_axis(None)
    return df


def load_yelp (url=PATH_YELP_DATA):
    """Sets CAMIS as index."""
    df = pd.read_csv(url, encoding='ISO-8859-1')
    df['zip_code'] = df['zip_code'].astype(np.int64)
    df.set_index('CAMIS', inplace=True, drop=False)
    if len(df.index) < 9500:
        raise RuntimeError('Yelp DF contains < 9,500 rows.')
    df.index.name = None
    df.rename_axis(None)
    return df


def load_complete_data ():
    inspections = load_inspections()
    yelp = load_yelp()
    result = pd.merge(inspections, yelp, how='left', on='CAMIS')
    return result

In [4]:
data = load_complete_data()

In [5]:
n, p = data.shape
print(f"Number of observations: {n}")
print(f"Number of predictors: {p}")

Number of observations: 157955
Number of predictors: 33


In [6]:
data.dtypes

CAMIS                      int64
DBA                       object
BORO                      object
BUILDING                  object
STREET                    object
ZIPCODE                    int64
PHONE                     object
CUISINE.DESCRIPTION       object
INSPECTION.DATE           object
ACTION                    object
VIOLATION.CODE            object
VIOLATION.DESCRIPTION     object
CRITICAL.FLAG             object
SCORE                    float64
GRADE                     object
GRADE.DATE                object
RECORD.DATE                int64
INSPECTION.TYPE           object
id                        object
name                      object
url                       object
phone                    float64
latitude                 float64
longitude                float64
review_count               int64
price                    float64
rating                   float64
transactions              object
categories                object
address                   object
city      

In [7]:
data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE.DESCRIPTION,INSPECTION.DATE,ACTION,VIOLATION.CODE,VIOLATION.DESCRIPTION,CRITICAL.FLAG,SCORE,GRADE,GRADE.DATE,RECORD.DATE,INSPECTION.TYPE,id,name,url,phone,latitude,longitude,review_count,price,rating,transactions,categories,address,city,state,zip_code
0,41653607,CHOP'T,MANHATTAN,18,EAST 23 STREET,10010,6467557837,Salads,2/21/2017,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,13.0,A,2/21/2017,42927,Cycle Inspection / Re-inspection,chopt-creative-salad-co-new-york-8,Chopt Creative Salad Co.,https://www.yelp.com/biz/chopt-creative-salad-...,1.646756e+10,40.740670,-73.988240,141,2.0,3.0,NaN,['salad'],18 East 23rd St,New York,NY,10010
1,41087519,REDEMPTION GRILL,MANHATTAN,1003,2 AVENUE,10022,2123194545,American,8/31/2016,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,26.0,B,8/31/2016,42927,Cycle Inspection / Re-inspection,redemption-nyc-new-york,Redemption NYC,https://www.yelp.com/biz/redemption-nyc-new-yo...,1.212319e+10,40.756966,-73.967326,289,2.0,3.0,"['delivery', 'pickup', 'restaurant_reservation']","['sportsbars', 'breakfast_brunch']",1003 2nd Ave,New York,NY,10022
2,41322119,AMIGOS,MANHATTAN,2888,BROADWAY,10025,2128641143,Mexican,9/19/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,27.0,NaN,NaN,42927,Cycle Inspection / Compliance Inspection,amigos-new-york,Amigos,https://www.yelp.com/biz/amigos-new-york?adjus...,1.212864e+10,40.805540,-73.965308,259,2.0,3.0,"['delivery', 'pickup', 'restaurant_reservation']",['mexican'],2888 Broadway,New York,NY,10025
3,41574702,WENDY'S,BRONX,4330,BOSTON ROAD,10475,7183258750,American,1/26/2015,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,8.0,A,1/26/2015,42927,Cycle Inspection / Initial Inspection,wendys-bronx-2,Wendy's,https://www.yelp.com/biz/wendys-bronx-2?adjust...,1.718326e+10,40.889590,-73.819980,10,1.0,2.0,NaN,"['hotdogs', 'burgers']",4330 Boston Rd,Bronx,NY,10475
4,40551234,THE GIN MILL,MANHATTAN,442,AMSTERDAM AVENUE,10024,2125809080,American,42708,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,9.0,A,42708,42927,Cycle Inspection / Initial Inspection,the-gin-mill-new-york,The Gin Mill,https://www.yelp.com/biz/the-gin-mill-new-york...,1.212581e+10,40.784760,-73.977540,325,2.0,3.5,NaN,"['tradamerican', 'sportsbars', 'cocktailbars']",442 Amsterdam Ave,New York,NY,10024


## Data Cleaning

In [8]:
# Update column names with "_" separator
data.columns = [c.replace(".", "_").lower() for c in data.columns]

In [9]:
cols = ['camis', 'zipcode', 'violation_code', 'cuisine_description', 'transactions', 'categories']
for c in cols:
    cnt = data[c].nunique()
    print("{}: {}".format(c, cnt))

camis: 9509
zipcode: 196
violation_code: 98
cuisine_description: 83
transactions: 13
categories: 3446


In [10]:
# Function to fix date formats 
from datetime import date, timedelta
def get_excel_date(row):
    if row==row:
        try:
            if len(row)==5:
                return date(1900, 1, 1) + timedelta(int(row))
            else:
                return pd.to_datetime(row).strftime('%Y-%m-%d')
        except:
            None
    else:
        return None

In [11]:
# Fix formats of date columns
date_cols = ['inspection_date', 'grade_date', 'record_date']
for col in date_cols:
    data[col] = data[col].apply(get_excel_date)
    
# Create inspection date variables
data['inspection_year'] = pd.DatetimeIndex(data['inspection_date']).year
data['inspection_month'] = pd.DatetimeIndex(data['inspection_date']).month

In [12]:
# Drop observations with no score 
data = data[pd.notnull(data.score)]

# Drop observations with a negative score
data = data[data.score >= 0]
      
# Replace missing values with "NULL" 
#   This causes issues when doing group by below
data.fillna("NULL", inplace=True)

# Number of unique observations (restaurant, inspection date)
count = len(data[["camis", "inspection_date"]].drop_duplicates())
print("Number of unique observations: {}".format(count))

Number of unique observations: 52472


In [13]:
model_cols = ['camis',
              'dba',
              'boro',
              'zipcode',
              'cuisine_description',
              'inspection_date',
              'inspection_year',
              'inspection_month',
              'score',
              'grade',
              'name',
              'latitude',
              'longitude',
              'review_count',
              'price',
              'rating',
              'transactions',
              'categories',
              'city']

# Group violations such that each observation in data is an individual inspection
grp = data.groupby(model_cols, group_keys=True)['violation_code'].apply(list)
data = pd.DataFrame(grp.reset_index())

In [14]:
# Ensure observations are unique by camis and inspection date
# Take observation with lowest score if multiple entries exist

# Rank duplicate observations by score
g = data.sort_values(['camis', 'inspection_date', 'score']).groupby(['camis', 'inspection_date'])
data['RNK'] = g['score'].rank(method='first')

# Drop duplicates
n = len(data)
data = data.loc[data.RNK == 1, :]
data.drop('RNK', axis=1, inplace=True)

# Set index
data.set_index(['camis', 'inspection_date'], drop=False, verify_integrity=True, inplace=True)

In [15]:
data.head()

camis                    dba      boro  zipcode cuisine_description inspection_date  inspection_year  inspection_month  score grade                 name   latitude  longitude  review_count price  rating transactions                categories      city             violation_code
camis    inspection_date                                                                                                                                                                                                                                                                                           
30075445 2015-09-04       30075445  MORRIS PARK BAKE SHOP     BRONX    10462              Bakery      2015-09-04           2015.0               9.0    6.0     A  Morris Pk Bake Shop  40.848446 -73.856079            31     2     4.5         NULL  ['bakeries', 'desserts']     Bronx                      [06C]
         2016-02-18       30075445  MORRIS PARK BAKE SHOP     BRONX    10462              Bakery      2016-02-18           2016.0               2.0   10.0     A  Morris Pk Bake Shop  40.848446 -73.856079            31     2     4.5         NULL  ['bakeries', 'desserts']     Bronx                 [04L, 08A]
         2017-05-18       30075445  MORRIS PARK BAKE SHOP     BRONX    10462              Bakery      2017-05-18           2017.0               5.0    7.0     A  Morris Pk Bake Shop  40.848446 -73.856079            31     2     4.5         NULL  ['bakeries', 'desserts']     Bronx                 [10F, 06D]
30112340 2015-07-07       30112340                WENDY'S  BROOKLYN    11225          Hamburgers      2015-07-07           2015.0               7.0   12.0     A              Wendy's  40.662952 -73.961753            30     1     2.0         NULL    ['hotdogs', 'burgers']  Brooklyn                 [10B, 04A]
         2016-03-12       30112340                WENDY'S  BROOKLYN    11225          Hamburgers      2016-03-12           2016.0               3.0   48.0  NULL              Wendy's  40.662952 -73.961753            30     1     2.0         NULL    ['hotdogs', 'burgers']  Brooklyn  [10A, 06A, 06C, 08A, 04N]

#### Cuisine Descriptions

In [16]:
data.dtypes

camis                           int64
dba                            object
boro                           object
zipcode                         int64
cuisine_description            object
inspection_date        datetime64[ns]
inspection_year               float64
inspection_month              float64
score                         float64
grade                          object
name                           object
latitude                      float64
longitude                     float64
review_count                    int64
price                          object
rating                        float64
transactions                   object
categories                     object
city                           object
violation_code                 object
dtype: object

In [17]:
def fix_cuisine_format(row):

    mapping = {
        'Bottled beverages, including water, sodas, juices, etc.': 'Bottled_beverages',
        'CafÃ©/Coffee/Tea': 'Coffee_Tea',
        'Ice Cream, Gelato, Yogurt, Ices': 'IceCream_Gelato',
        'Juice, Smoothies, Fruit Salads': 'Juice_Smoothies',
        'Latin (Cuban, Dominican, Puerto Rican, South & Central American)': 'Latin',
        'Sandwiches/Salads/Mixed Buffet': 'Sandwiches',
        'Soups & Sandwiches': 'Sandwiches',
        'Tex-Mex': 'TexMex'
    }
    
    try:
        row = mapping[row]
    except:
        row = row.strip().replace("/", "_").replace(" ", "_")
    
    return row
        
data['cuisine_description'] = data['cuisine_description'].apply(fix_cuisine_format)

In [18]:
def fix_list_format(row):
    
    row = str(row)
    
    chars = ''''"[]'''
    for c in chars:
        row = row.strip().replace(c, "")
    row = row.replace(",", "|").replace(" ", "")
    
    return row

#### Transactions

In [19]:
# Clean values
data['transactions'] = data['transactions'].apply(fix_list_format)

# Get unique set of transactions
transactions = set()
for s in data.transactions:
    transactions.update(s.split('|'))

# Create separate columns for each unique value
for txn in transactions:
    data["trans_" + txn] = [1 if txn in t.split('|') else 0 for t in data.transactions]
    
# Drop column
data.drop('transactions', axis=1, inplace=True)

#### Categories

In [20]:
# Clean up categories column
data['categories'] = data['categories'].apply(fix_list_format)
print("Total number of category combinations: {}".format(data.categories.nunique()))

# Get set of unique categories
categories = set()
for s in data.categories:
    categories.update(s.split('|'))
print("Total number of unique categories: {}".format(len(categories)))

# Create column for each unique category and encode 
# Might need to refine this later to reduce dimensionality
for cat in categories:
    data["cat_" + cat] = [1 if cat in c.split('|') else 0 for c in data.categories]
    
# Drop column
data.drop('categories', axis=1, inplace=True)

Total number of category combinations: 3423
Total number of unique categories: 309


#### Violations

In [21]:
# Clean up categories column
data['violation_code'] = data['violation_code'].apply(fix_list_format)
print("Total number of violation combinations: {}".format(data.violation_code.nunique()))

# Get set of unique categories
violations = set()
for s in data.violation_code:
    violations.update(s.split('|'))
print("Total number of unique violations: {}".format(len(violations)))

# Create column for each unique category and encode 
#    Might need to refine this later to reduce dimensionality
for vcode in violations:
    data["violation_" + vcode] = [1 if vcode in v.split('|') else 0 for v in data.violation_code]
    
# Drop column
data.drop('violation_code', axis=1, inplace=True)

Total number of violation combinations: 17796
Total number of unique violations: 70


#### Restaurant Names

In [22]:
# Length of restuarant name in inspection data
data["restaurant_name_len"] = data["dba"].apply(len)

# Length of restaurant name in Yelp data
data["restaurant_name_len_yelp"] = data["name"].apply(len)

# Similarity of names in inspection data and Yelp data
data["restaurant_name_sim"] = data.apply(lambda row: distance.levenshtein(row["dba"], row["name"]), axis=1)

# Number of observations by restaurant 
df = data.dba.value_counts().reset_index()
df.columns = ['dba', 'restaurant_name_count']
data = pd.merge(data, df, how='left', on='dba')

# Create initial flag for restaurant chains (count > 50)
data["restaurant_name_chain"] = data['restaurant_name_count'].apply(lambda x: 1 if x > 50 else 0)

# Drop column
data.drop('name', axis=1, inplace=True)

#### Demographics by zipcode

In [23]:
# Import data
cen_data = pd.read_csv(PATH_DEMOGRAPHICS)

# Update column names
cen_data.columns =  ["cen_" + c.replace(" ", "_").lower() for c in cen_data.columns]
cen_data.columns.values[0] = 'zipcode'

# Merge onto base dataset
data = pd.merge(data, cen_data, how='left', on='zipcode')

#### Region

In [24]:
# Borough
##########

# Create dummy variables
col = 'boro'
df = pd.get_dummies(data[col], prefix=col)
data = data.join(df)
data.drop(col, axis=1, inplace=True)

# City
#########

# Group areas with low volumes
col = 'city'
counts = list(pd.value_counts(data[col]).index[:30])
data[col] = data[col].apply(lambda row: row if row in counts else 'Other')

# Create dummy variables
df = pd.get_dummies(data[col], prefix=col)
data = data.join(df)
data.drop(col, axis=1, inplace=True)

In [25]:
# Clean up column names (lowercase and no spaces)
data.columns = [c.replace(".", "_").lower() for c in data.columns]

# Replace NULL with NA again
data.replace("NULL", np.nan, inplace=True)

# Impute missing values (categorical)
cols = ['rating', 'price']
imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
data[cols] = imp.fit_transform(data[cols])

# Census values
imp = SimpleImputer(missing_values=np.nan, strategy="mean")
cols = [c for c in data.columns if c[:3] == "cen"]
data[cols] = imp.fit_transform(data[cols])

# Reorder columns
cols = list(data.columns)
target_cols = ['grade', 'score']
for c in target_cols:
    cols.remove(c)
    cols.append(c)

data = data[cols]

In [26]:
#Export data to csv
#data.to_csv('../data/model_data.csv')

### Please proceed to the next jupyter notebook labeled "02_eda_and_modeling" located in this same folder for the second half of this project